In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
import math


### Ventana móvil

Ventana móvil con 2 fases.
1. Marca la "frontera" de los periodos válidos mirando hacia atrás. Marcará un 1 si el periodo de 365 días anterior es válido.
2. Refrescar el periodo de 365 días anterior a cada "flag" que tenga un 1, para marcar que el periodo ya es válido.

In [2]:

def set_rowling_window(df:pd.DataFrame, window_size= 365 * 24, nulls_tolerance= 15 * 24 ) -> pd.DataFrame:
    """
    window_size: tamaño de la ventana para el cálculo de la media móvil
    nulls_tolerance: número de nulos tolerados en la ventana
    """
    window_size = 365 * 24  
    nulls_tolerance = 15 * 24 

    #Nos saltamos la columna de fecha e identificacion de la base
    for col in df.columns:
        if df[col].dtype != np.float64 and df[col].dtype != np.int64:        
            continue
        print(f'{col} is a number')
        df[f'{col}_{station}'] = df[col].rolling(window=window_size, min_periods=1).apply(
            lambda x: 0 if x.isnull().sum() > nulls_tolerance or len(x) < window_size else 1
        )
        df[f'{col}_{station}'] = df[f'{col}_{station}'].replace({np.nan: 0})
        df[f'{col}_{station}'] = df[f'{col}_{station}'].astype(np.int8)

    return df

### Recalificación de flags

In [3]:

def recalificacion_flags(df: pd.DataFrame, window_size=365 * 24) -> pd.DataFrame:
    for col in df.columns:
        referenced_col = col.split('_')[0]
        
        if col.endswith(station):        
            print(f'{col} referenciando {referenced_col} va a ser reasignada ')
            for i in range(len(df[col])):
                if df.iloc[i, df.columns.get_loc(col)] == 1:  # Acceder al valor de la columna con iloc
                    start_idx = max(0, i - window_size + 1)  
                    
                    # Buscar el primer índice no nulo en referenced_col dentro de la ventana
                    for j in range(start_idx, i + 1):
                        if not pd.isna(df.iloc[j, df.columns.get_loc(referenced_col)]):
                            df.iloc[j:i+1, df.columns.get_loc(col)] = 1  # Recalificar desde el primer índice no nulo hasta el índice actual
                            break  # Terminar la búsqueda una vez encontrado el primer valor no nulo
        else:
            print(f'{col} NO va a ser reasignada ---> ')
    return df

### Visualización de los datos

In [4]:

def plot_all_data(df): 
    df['date'] = pd.to_datetime(df['date'])
    print ('Obteniendo gráficas generales')
    num_cols = len(df.columns) - 1  # Restamos 1 para excluir la columna 'fhora'
    num_rows = math.ceil(num_cols / 4)  # Redondea hacia arriba para obtener el número de filas necesarias

    fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, 5 * num_rows))  # Ajusta el tamaño de la figura según el número de filas


    # Aplanar el array de ejes para poder iterar sobre él
    axes = axes.flatten()

    # Iterar sobre las columnas (excluyendo 'fhora') y generar gráficos
    for ax, col in zip(axes, df.columns.drop('date')):
        print(f'Procesando gráfica de {col}')
        ax.plot(df['date'], df[col])
        ax.set_xlabel('date')
        ax.set_ylabel(col)
        ax.set_title(f'{col} a lo date de fhora')

    # Eliminar los ejes sobrantes si el número de columnas no es múltiplo de 4
    if num_cols % 4:
        for ax in axes[num_cols:]:
            ax.remove()

    plt.tight_layout()
    plt.savefig(f'resource/flagged_{station}.png')

    plt.show()

### Generación del heatmap

In [5]:
def plot_heatmap(df):
    df['date'] = pd.to_datetime(df['date'])
    df_flags = df[[col for col in df.columns if station in col]]
    df_flags['date'] = df['date']

    df_flags.set_index('date', inplace=True)
    df_flags.index = pd.to_datetime(df_flags.index)
    df_flags.index = df_flags.index.to_period('M') 

    num_cols = len(df_flags.columns)-1

    num_rows = math.ceil(num_cols / 3)  
    print ('Obteniendo mapa de calor')
    print(num_cols, num_rows)

    fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(12, 8 * num_rows))  # Ajusta el tamaño de la figura según el número de filas

    # Aplanar el array de ejes para poder iterar sobre él
    axes = axes.flatten()

    # Iterar sobre las columnas y generar gráficos
    for ax, col in zip(axes, df_flags.columns):
        if col != 'date':
            print(f'{col} procede al mapa de calor')
            sns.heatmap(df_flags[[col]], cbar=False, cmap='viridis', ax=ax)
            ax.set_title(col)
        

    # Eliminar los ejes sobrantes si el número de columnas no es múltiplo de 3
    if num_cols % 3:
        for ax in axes[num_cols:]:
            ax.remove()

    plt.tight_layout()
    plt.savefig(f'resource/heatmap_{station}_window.png')
    plt.show()

    df.head()

### Carga de dataframes y ejecución

In [6]:

path = './data/'

WINDOW_SIZE = 365 * 24
NULLS_TOLERANCE = 15 * 24

for dirpath, dirnames, filenames in os.walk(path):
    print(f'Found directory: {dirpath}')
    for dirname in dirnames:
        station = dirname.split('_')[1]
        input_path = f'{station}_clean.csv'
        
        try:
            df = pd.read_csv(path + dirname + '/'  + input_path)
            df.head()
            print(f'Nombre del input csv: {input_path}')
        except Exception as e:
            print(f'ERROR EN LA LECTURA DE LA ESTACIÓN {station} ---> {e}')
            continue

        df = set_rowling_window(df, window_size=WINDOW_SIZE, nulls_tolerance=NULLS_TOLERANCE)
        df = recalificacion_flags(df, window_size=WINDOW_SIZE)
        df.to_csv(f'./flagged_data/{station}_flagged.csv', index=False)
        


Found directory: ./data/
Nombre del input csv: dismal_clean.csv
temp is a number
pres is a number
vel is a number
dir is a number
hr is a number
temp NO va a ser reasignada ---> 
pres NO va a ser reasignada ---> 
vel NO va a ser reasignada ---> 
dir NO va a ser reasignada ---> 
hr NO va a ser reasignada ---> 
date NO va a ser reasignada ---> 
temp_dismal referenciando temp va a ser reasignada 
pres_dismal referenciando pres va a ser reasignada 
vel_dismal referenciando vel va a ser reasignada 
dir_dismal referenciando dir va a ser reasignada 
hr_dismal referenciando hr va a ser reasignada 
Nombre del input csv: ferraz_clean.csv
date NO va a ser reasignada ---> 
hr NO va a ser reasignada ---> 
prec NO va a ser reasignada ---> 
pres NO va a ser reasignada ---> 
temp NO va a ser reasignada ---> 
dir NO va a ser reasignada ---> 
vel NO va a ser reasignada ---> 
Nombre del input csv: gdc_clean.csv
temp is a number
skt is a number
dir is a number
vel is a number
hr is a number
prec is a numb

### Obtener las gráficas de todos los dataset


In [11]:

path = './flagged_data/'
filenames = os.listdir(path)

for filename in filenames:
    print(f'Found directory: {filename}')
    file_path = os.path.join(path, filename)
    station = filename.split('_')[0]
    try:
        df = pd.read_csv(file_path)
        df.head()
        print(f'Nombre del input csv: {file_path}')
    except Exception as e:
        print(f'ERROR EN LA LECTURA DE LA ESTACIÓN {station} ---> {e}')
        continue

    try:
        plot_all_data(df)
        plot_heatmap(df)
    except Exception as e:
        print(f'ERROR AL OBTENER LA GRÁFICA ---> {e}')
        continue

    

Found directory: dismal_flagged_test.csv
Nombre del input csv: ./data/flagged_data/dismal_flagged_test.csv
Obteniendo gráficas generales
Procesando gráfica de temp
Procesando gráfica de pres
Procesando gráfica de vel
Procesando gráfica de dir
Procesando gráfica de hr
Procesando gráfica de temp_dismal
Procesando gráfica de pres_dismal
Procesando gráfica de vel_dismal
Procesando gráfica de dir_dismal
Procesando gráfica de hr_dismal


KeyboardInterrupt: 